In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "technology.pdf"
loader = PyMuPDFLoader(file_path)

In [2]:
docs = loader.load()
docs

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-07-12T01:04:21+03:00', 'source': 'technology.pdf', 'file_path': 'technology.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': 'Osama Ahmed', 'subject': '', 'keywords': '', 'moddate': '2025-07-12T01:04:21+03:00', 'trapped': '', 'modDate': "D:20250712010421+03'00'", 'creationDate': "D:20250712010421+03'00'", 'page': 0}, page_content='NovaTech Electronics Product Catalog — July 2025 Edition \nWelcome to the official product catalog of NovaTech Electronics. We specialize in high-\nperformance consumer tech, offering the best in smartphones, laptops, tablets, smartwatches, \nheadphones, and accessories. Below you\'ll find descriptions, specs, prices, and availability for \nour latest lineup. \n \nSMARTPHONES \n1. Nova S9 Max \nModel: NS9-2025-A1 \nA flagship smartphone with a stunning 6.7" AMOLED display, Snapdragon 8 Gen 2 \nprocessor, 12GB RAM, and a 256GB sto

In [3]:
raw_texts = [doc.page_content for doc in docs]


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
# with open("state_of_the_union.txt") as f:
    #state_of_the_union = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    add_start_index=True,
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)
texts = text_splitter.create_documents(raw_texts)
print(texts[0].page_content)


NovaTech Electronics Product Catalog — July 2025 Edition 
Welcome to the official product catalog of NovaTech Electronics. We specialize in high-
performance consumer tech, offering the best in smartphones, laptops, tablets, smartwatches, 
headphones, and accessories. Below you'll find descriptions, specs, prices, and availability for 
our latest lineup. 
 
SMARTPHONES 
1. Nova S9 Max 
Model: NS9-2025-A1 
A flagship smartphone with a stunning 6.7" AMOLED display, Snapdragon 8 Gen 2


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="")

In [8]:
from langchain_chroma import Chroma

db = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="data/chroma_langchain_db",
    collection_name="my_chunks"
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

retriever = db.as_retriever()
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    google_api_key=""
)
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")],
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
result = chain.invoke({"input": input("Enter Your Question: ")})

In [14]:
print(result["answer"])

No, I do not have any information about "gebally" in the provided context. The given text only describes various Nova products like the NovaBook Creator, Nova Watch Sport, NovaTab for Business, and details two smartphone models.
